In [1]:
# Cell 1 — install & imports
#!pip install -q econml scikit-learn xgboost pandas matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from econml.dml import LinearDML
import warnings
warnings.filterwarnings("ignore")
RNG = 123


c:\Users\nahom\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


df = pd.read_csv('./last_month_data (2).csv')
print(df.shape)
df.head(3)


(153510, 26)


,period_normalized_date,category,brand,product_name,sub_segment,promotion,uom,item_code,sales_units,price_per_sales_unit,...,is_holiday,month,day_of_week,day_of_month,rolling_3day_mean,rolling_7day_mean,rolling_30day_mean,rolling_3day_std,rolling_7day_std,rolling_30day_std
0,2023-01-01,TOTAL COFFEE,NESCAFE,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),INSTANT MIXES,NO PROMO,KGS-LTRS,2.000040e+12,156.862570,6.962713,...,1,1,6,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,TOTAL COFFEE,NESCAFE,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),INSTANT MIXES,NO PROMO,KGS-LTRS,2.000040e+12,100.990090,6.962713,...,0,1,0,2,156.86257,156.86257,156.86257,NaN,NaN,NaN
2,2023-01-03,TOTAL COFFEE,NESCAFE,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),INSTANT MIXES,NO PROMO,KGS-LTRS,2.000040e+12,96.937669,6.962713,...,0,1,1,3,128.92633,128.92633,128.92633,39.50781,39.50781,39.50781


In [3]:
# Cell 3 — confirm columns & peek
print("Columns in dataframe:", df.columns.tolist())
print("\nQuick peek:")
display(df[['product_name','price_per_sales_unit','sales_units','promotion','month']].head())


Columns in dataframe: ['period_normalized_date', 'category', 'brand', 'product_name', 'sub_segment', 'promotion', 'uom', 'item_code', 'sales_units', 'price_per_sales_unit', 'price_per_sales_per_uom', 'sales_value', 'sales_per_uom', 'emirate', 'store_type', 'is_weekend', 'is_holiday', 'month', 'day_of_week', 'day_of_month', 'rolling_3day_mean', 'rolling_7day_mean', 'rolling_30day_mean', 'rolling_3day_std', 'rolling_7day_std', 'rolling_30day_std']

Quick peek:


,product_name,price_per_sales_unit,sales_units,promotion,month
0,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),6.962713,156.862570,NO PROMO,1
1,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),6.962713,100.990090,NO PROMO,1
2,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),6.962713,96.937669,NO PROMO,1
3,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),6.962713,94.467248,NO PROMO,1
4,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),6.962713,86.266077,NO PROMO,1


In [4]:
COL_PRICE = 'price_per_sales_unit'
COL_QTY = 'sales_units'
COL_PRODUCT = 'product_name'

df = df.copy()

# convert to numeric, force errors to NaN
df[COL_PRICE] = pd.to_numeric(df[COL_PRICE], errors='coerce')
df[COL_QTY] = pd.to_numeric(df[COL_QTY], errors='coerce')

# now filter
before = len(df)
df = df[df[COL_PRICE] > 0]
df = df[df[COL_QTY] >= 0]
after = len(df)

print(f"Rows before: {before}  |  after cleaning: {after}")


Rows before: 153510  |  after cleaning: 153510


In [5]:
# Cell 5 — log transforms (log-log model)
import numpy as np
eps = 1e-6
df['T'] = np.log(df[COL_PRICE].astype(float) + eps)   # treatment: log(price)
df['Y'] = np.log(df[COL_QTY].astype(float) + eps)     # outcome: log(quantity)
display(df[['T','Y']].describe().T)


,count,mean,std,min,25%,50%,75%,max
T,153510.0,0.486360,1.006807,-0.753256,-0.571822,0.454745,1.326043,1.974629
Y,153510.0,1.699342,2.851487,-7.237986,0.413522,2.376241,3.713781,7.183560


In [ ]:
# Cell 6 — covariates (we use sensible defaults you have)
covariates = [
    'promotion', 'month', 'day_of_week', 'is_weekend', 'is_holiday',
    'category', 'brand', 'store_type',
    'rolling_7day_mean', 'rolling_7day_std'
]

# keep only existing covariates
covariates = [c for c in covariates if c in df.columns]
print("Using covariates:", covariates)

X = df[covariates].copy()

# Handle NaN in promotion
if 'promotion' in X.columns:
    X['promotion'] = X['promotion'].fillna('NO PROMO')

# convert object/category columns to dummies (drop_first to reduce columns)
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
print("Categorical covariates:", cat_cols)
if len(cat_cols) > 0:
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

X = X.fillna(0)
print("Final X shape:", X.shape)
print("Feature columns:", X.columns.tolist())


Using covariates: ['promotion', 'month', 'day_of_week', 'is_weekend', 'is_holiday', 'category', 'brand', 'store_type', 'rolling_7day_mean', 'rolling_7day_std']
Categorical covariates: ['promotion', 'category', 'brand', 'store_type']
Final X shape: (153510, 18)


In [7]:
# Cell 7 — check price variation (very important)
print("Total unique prices:", df[COL_PRICE].nunique())
pv = df.groupby(COL_PRODUCT)[COL_PRICE].nunique()
print("\nPer-product price unique counts summary:")
print(pv.describe())

one_price = pv[pv==1].shape[0]
print(f"\nNumber of products with only 1 unique price: {one_price}")


Total unique prices: 120

Per-product price unique counts summary:
count     5.0
mean     24.0
std       0.0
min      24.0
25%      24.0
50%      24.0
75%      24.0
max      24.0
Name: price_per_sales_unit, dtype: float64

Number of products with only 1 unique price: 0


In [8]:
# Cell 8 — train LinearDML (Double ML)
from xgboost import XGBRegressor
from econml.dml import LinearDML

Y = df['Y'].values
T = df['T'].values
X_arr = X.values

# Using XGBoost instead of RandomForest
model_y = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=123, n_jobs=-1)
model_t = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=123, n_jobs=-1)

est = LinearDML(model_y=model_y, model_t=model_t,
                discrete_treatment=False, cv=3, random_state=123)

print("Fitting model (may take a minute)...")
est.fit(Y, T, X=X_arr)
print("Model fitted.")


Fitting model (may take a minute)...
Model fitted.


In [9]:
# Cell 9 — results: average elasticity + per-product summary (fixed version)

ate = est.ate(X=X_arr)

# safe CI computation
try:
    ate_low, ate_high = est.ate_interval(X=X_arr)
except Exception as e:
    print("Skipping confidence interval due to:", e)
    ate_low, ate_high = (np.nan, np.nan)

hetero = est.effect(X_arr)   # elasticity per row

print(f"\nAverage price elasticity (ATE) = {ate:.4f}")
if not np.isnan(ate_low):
    print(f"95% CI = ({ate_low:.4f}, {ate_high:.4f})")
print("Heterogeneous elasticity sample - mean:", hetero.mean(), " std:", hetero.std())

# attach and summarize by product
df['elasticity'] = hetero
prod_summary = df.groupby(COL_PRODUCT)['elasticity'].agg(['mean','std','count']).reset_index().sort_values('mean')
display(prod_summary.head(20))

# save summary
prod_summary.to_csv('product_elasticity_summary.csv', index=False)
print("\nSaved: product_elasticity_summary.csv — you can download from the left 'Files' panel.")


Skipping confidence interval due to: loop of ufunc does not support argument 0 of type float which has no callable sqrt method

Average price elasticity (ATE) = -0.2327
Heterogeneous elasticity sample - mean: -0.23270531891045854  std: 0.19250336438608318


,product_name,mean,std,count
3,NESTLE NESQUIK 330GR(C) BOX,-0.546543,0.101801,30702
2,NESTLE CHOCAPIC C/B 25GR (C) WRP,-0.202892,0.093544,30702
4,PURINA FRISK.CHICKEN IN GRAVY JUNI.85G S,-0.170898,0.122202,30702
1,NESCAFE LATTE 240ML TIN,-0.154605,0.090797,30702
0,NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM),-0.088589,0.113691,30702



Saved: product_elasticity_summary.csv — you can download from the left 'Files' panel.


In [ ]:
import joblib

# Save model
joblib.dump(est, 'price_elasticity_model.pkl')
print("Model saved as price_elasticity_model.pkl")

# Save feature names for later use
feature_info = {
    'feature_names': X.columns.tolist(),
    'num_features': len(X.columns)
}
joblib.dump(feature_info, 'elasticity_feature_info.pkl')
print("Feature info saved as elasticity_feature_info.pkl")


Model saved as price_elasticity_model.pkl


In [19]:
import pandas as pd
import numpy as np
import joblib

eps = 1e-6
COL_PRICE = 'price_per_sales_unit'
COL_QTY = 'sales_units'
COL_PRODUCT = 'product_name'

# Load your trained model
est = joblib.load('price_elasticity_model.pkl')

# log transforms
df['T'] = np.log(df[COL_PRICE].astype(float) + eps)
df['Y'] = np.log(df[COL_QTY].astype(float) + eps)

# --- Prepare covariates exactly like training ---
covariates = [
    'promotion', 'month', 'day_of_week', 'is_weekend', 'is_holiday',
    'category', 'brand', 'store_type',
    'rolling_7day_mean', 'rolling_7day_std'
]
covariates = [c for c in covariates if c in df.columns]

X = df[covariates].copy()

cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
if len(cat_cols) > 0:
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

X = X.fillna(0)

# Align columns with model
if hasattr(est, 'feature_names_in_'):
    for col in est.feature_names_in_:
        if col not in X.columns:
            X[col] = 0
    X = X[est.feature_names_in_]

X_arr = X.values

# Compute per-product mean elasticity
df['elasticity'] = est.effect(X_arr)
prod_elasticity = df.groupby(COL_PRODUCT)['elasticity'].mean().to_dict()
prod_price = df.groupby(COL_PRODUCT)[COL_PRICE].mean().to_dict()
prod_qty = df.groupby(COL_PRODUCT)[COL_QTY].mean().to_dict()

# Check price variation per product
price_variation = df.groupby(COL_PRODUCT)[COL_PRICE].nunique().to_dict()

# --- Interactive selection ---
products = list(prod_elasticity.keys())[:5]  # first 5 products
print("Available products:")
for i, p in enumerate(products, start=1):
    print(f"{i}. {p}")

choice = int(input("Choose a product by number (1-5): "))
product_name = products[choice-1]

new_price = float(input(f"Set a price for '{product_name}': "))

# Get elasticity and make sure negative
elasticity = -abs(prod_elasticity[product_name])  # force negative

# Warn if product has very low price variation
if price_variation[product_name] <= 2:
    print("⚠ Warning: this product has very low price variation; elasticity may be unreliable.")

# Calculate predicted quantity using log-log elasticity
old_price = prod_price[product_name]
base_qty = prod_qty[product_name]
predicted_qty = base_qty * (new_price / old_price) ** elasticity

# Calculate elasticity index at new price
elasticity_index = elasticity * (new_price / old_price)

print(f"\nElasticity index for {product_name} at price {new_price}: {elasticity_index:.4f}")
print(f"Predicted quantity at price {new_price}: {predicted_qty:.2f} units")


Available products:
1. NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM)
2. NESCAFE LATTE 240ML TIN
3. NESTLE CHOCAPIC C/B 25GR (C) WRP
4. NESTLE NESQUIK 330GR(C) BOX
5. PURINA FRISK.CHICKEN IN GRAVY JUNI.85G S

Elasticity index for NESCAFE 3IN1 CLASSIC 20GX24 BOX (CM) at price 30.0: -0.3970
Predicted quantity at price 30.0: 64.52 units
